In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models,Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,MaxPooling1D,Conv1D
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers import Flatten, Input, concatenate
import scipy as sp
# !pip install spafe==0.2.0
import spafe
from spafe.features.gfcc import gfcc
from scipy.stats import kurtosis,skew,mode,gstd,describe,iqr,gmean,hmean,median_abs_deviation,variation,tstd,gstd,moment,entropy
import librosa, librosa.display
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics as skm
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam 

In [3]:
df=pd.read_csv('/content/drive/MyDrive/Eusipco Paper/mvd_gfcc.csv')
X_train=np.load('/content/drive/MyDrive/Eusipco Paper/mvd_gfcc.npy',allow_pickle=True)
Y_train=np.array(df['vehicle'].tolist())

In [4]:
df['vehicle'].value_counts()

M    4488
T    4308
N    4100
C    4020
Name: vehicle, dtype: int64

In [5]:
def create_mlp(dim, regularizer=None):
  
    model = Sequential()
    model.add(Dense(13, input_dim=dim, activation="relu", kernel_regularizer=regularizer))
    return model

def create_cnn(width, height, filters=(64,32), regularizer=None):

    inputShape = (height, width)
    inputs = Input(shape=inputShape)
 
    for (i, f) in enumerate(filters):

        if i == 0:
            x = inputs

        x = Conv1D(f, (3), padding="same")(x)
        x = MaxPooling1D(pool_size=(2))(x)
    
    x = Flatten()(x)
    x = Dense(13,activation="relu", kernel_regularizer=regularizer)(x)
    model = Model(inputs, x)
    return model      

In [14]:
strtfdKFold = StratifiedKFold(n_splits=5)
kfold = strtfdKFold.split(X_train, Y_train)

for k, (train, test) in enumerate(kfold):

  X_tr=X_train[train,:]
  Y_tr=Y_train[train]
  X_te=X_train[test,:]
  Y_te=Y_train[test]
  
  
  labelencoder=LabelEncoder()
  y_train=to_categorical(labelencoder.fit_transform(Y_tr))
  labelencoder=LabelEncoder()
  y_test=to_categorical(labelencoder.fit_transform(Y_te))
  globalx_train=[]
  localx_train=[]
  globalx_test=[]
  localx_test=[] 

  for i in range(len(X_tr)):
    globalx_train.append(X_tr[i][1])
    localx_train.append(X_tr[i][0])

  for i in range(len(X_te)):
    globalx_test.append(X_te[i][1])
    localx_test.append(X_te[i][0])

  globalx_train=np.array(globalx_train)
  localx_train=np.array(localx_train)
  globalx_test=np.array(globalx_test)
  localx_test=np.array(localx_test)

  mlp = create_mlp(13)
  cnn = create_cnn(40,128)
  
  combinedInput = concatenate([mlp.output, cnn.output])

  x=Dense(20, activation="relu")(combinedInput)
  x = Dense(100, activation="relu")(x)
  x = Dense(4, activation="softmax")(x)

  model1 = Model(inputs=[mlp.input, cnn.input], outputs=x)


  opt = Adam(learning_rate=1e-3, decay=1e-3 / 200)
  model1.compile(loss="categorical_crossentropy", metrics=['acc'], optimizer=opt)
  earlyStopping = EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min')
  mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
  reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4, mode='min')

  model1_history = model1.fit( 
  [globalx_train, localx_train], 
  np.array(y_train), 
  validation_data=([globalx_test, localx_test],np.array(y_test)), 
  epochs=50,callbacks=[mcp_save, reduce_lr_loss,earlyStopping])
  
  print('\n')
  print("fold",k)
  print("Accuracy of the model on Training Data is - " , model1.evaluate([globalx_train,localx_train], y_train)[1]*100 , "%")
  print("Accuracy of the model on Testing Data is - " , model1.evaluate([globalx_test, localx_test], y_test)[1]*100 , "%")
  print('\n')

  y_predict=(model1.predict([globalx_test, localx_test]) > 0.5).astype("int32")
  y_true=y_test
  cm= skm.multilabel_confusion_matrix(y_true, y_predict)
  print(skm.classification_report(y_true,y_predict))
  print('\n')

Epoch 1/50
423/423 [==============================] - 10s 21ms/step - loss: 0.5099 - acc: 0.8018 - val_loss: 0.3824 - val_acc: 0.8549 - lr: 0.0010
Epoch 2/50
423/423 [==============================] - 10s 24ms/step - loss: 0.3141 - acc: 0.8838 - val_loss: 0.3459 - val_acc: 0.8644 - lr: 0.0010
Epoch 3/50
423/423 [==============================] - 10s 23ms/step - loss: 0.2833 - acc: 0.8968 - val_loss: 0.2994 - val_acc: 0.8883 - lr: 0.0010
Epoch 4/50
423/423 [==============================] - 8s 18ms/step - loss: 0.2549 - acc: 0.9049 - val_loss: 0.2814 - val_acc: 0.8921 - lr: 0.0010
Epoch 5/50
423/423 [==============================] - 10s 23ms/step - loss: 0.2314 - acc: 0.9143 - val_loss: 0.2681 - val_acc: 0.9007 - lr: 0.0010
Epoch 6/50
423/423 [==============================] - 8s 20ms/step - loss: 0.2044 - acc: 0.9257 - val_loss: 0.2323 - val_acc: 0.9116 - lr: 0.0010
Epoch 7/50
423/423 [==============================] - 9s 20ms/step - loss: 0.1796 - acc: 0.9371 - val_loss: 0.2512 - val

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      0.93      0.94       804
           1       0.94      0.95      0.95       898
           2       0.98      1.00      0.99       820
           3       0.97      0.96      0.97       862

   micro avg       0.96      0.96      0.96      3384
   macro avg       0.96      0.96      0.96      3384
weighted avg       0.96      0.96      0.96      3384
 samples avg       0.96      0.96      0.96      3384



Epoch 1/50
423/423 [==============================] - 11s 22ms/step - loss: 0.5077 - acc: 0.8001 - val_loss: 0.3282 - val_acc: 0.8779 - lr: 0.0010
Epoch 2/50
423/423 [==============================] - 10s 23ms/step - loss: 0.3244 - acc: 0.8801 - val_loss: 0.2852 - val_acc: 0.8897 - lr: 0.0010
Epoch 3/50
423/423 [==============================] - 9s 22ms/step - loss: 0.2805 - acc: 0.8967 - val_loss: 0.2812 - val_acc: 0.8912 - lr: 0.0010
Epoch 4/50
423/423 [==============================] - 8s 19ms/step - 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/50
423/423 [==============================] - 11s 23ms/step - loss: 0.4770 - acc: 0.8172 - val_loss: 0.3392 - val_acc: 0.8723 - lr: 0.0010
Epoch 2/50
423/423 [==============================] - 8s 19ms/step - loss: 0.3063 - acc: 0.8869 - val_loss: 0.3175 - val_acc: 0.8794 - lr: 0.0010
Epoch 3/50
423/423 [==============================] - 9s 20ms/step - loss: 0.2840 - acc: 0.8934 - val_loss: 0.2671 - val_acc: 0.9042 - lr: 0.0010
Epoch 4/50
423/423 [==============================] - 10s 23ms/step - loss: 0.2469 - acc: 0.9077 - val_loss: 0.2627 - val_acc: 0.9078 - lr: 0.0010
Epoch 5/50
423/423 [==============================] - 10s 23ms/step - loss: 0.2251 - acc: 0.9181 - val_loss: 0.2545 - val_acc: 0.9033 - lr: 0.0010
Epoch 6/50
423/423 [==============================] - 10s 23ms/step - loss: 0.2000 - acc: 0.9277 - val_loss: 0.1978 - val_acc: 0.9282 - lr: 0.0010
Epoch 7/50
423/423 [==============================] - 9s 20ms/step - loss: 0.1755 - acc: 0.9404 - val_loss: 0.1904 - val

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      0.95      0.95       804
           1       0.96      0.96      0.96       897
           2       0.99      0.99      0.99       820
           3       0.97      0.97      0.97       862

   micro avg       0.97      0.97      0.97      3383
   macro avg       0.97      0.97      0.97      3383
weighted avg       0.97      0.97      0.97      3383
 samples avg       0.97      0.97      0.97      3383



Epoch 1/50
423/423 [==============================] - 12s 24ms/step - loss: 0.4828 - acc: 0.8153 - val_loss: 0.3256 - val_acc: 0.8815 - lr: 0.0010
Epoch 2/50
423/423 [==============================] - 9s 22ms/step - loss: 0.3141 - acc: 0.8821 - val_loss: 0.3144 - val_acc: 0.8812 - lr: 0.0010
Epoch 3/50
423/423 [==============================] - 8s 18ms/step - loss: 0.2744 - acc: 0.9002 - val_loss: 0.2574 - val_acc: 0.9072 - lr: 0.0010
Epoch 4/50
423/423 [==============================] - 9s 22ms/step - l

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      0.97      0.95       804
           1       0.96      0.96      0.96       898
           2       0.99      0.99      0.99       820
           3       0.98      0.96      0.97       861

   micro avg       0.97      0.97      0.97      3383
   macro avg       0.97      0.97      0.97      3383
weighted avg       0.97      0.97      0.97      3383
 samples avg       0.97      0.97      0.97      3383



Epoch 1/50
423/423 [==============================] - 10s 22ms/step - loss: 0.5820 - acc: 0.7537 - val_loss: 0.3276 - val_acc: 0.8868 - lr: 0.0010
Epoch 2/50
423/423 [==============================] - 8s 20ms/step - loss: 0.3285 - acc: 0.8815 - val_loss: 0.3071 - val_acc: 0.8859 - lr: 0.0010
Epoch 3/50
423/423 [==============================] - 10s 23ms/step - loss: 0.2861 - acc: 0.8980 - val_loss: 0.2623 - val_acc: 0.9033 - lr: 0.0010
Epoch 4/50
423/423 [==============================] - 8s 18ms/step - 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
dict(zip(labelencoder.classes_,range(len(labelencoder.classes_))))

{'C': 0, 'M': 1, 'N': 2, 'T': 3}